# Get Lines from TV Vid

Since synthetic images of rings from different angles can be generated, and certain aspects of the tokamak exhibits ring-like behavior, an assumption can be made that there can be a direct mapping between a processed real image and a synthetic image via ring creation. Different methods are employed here in order to attempt an extraction of the ring-like structures from bright emission areas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from pathlib import Path
from tqdm.notebook import tqdm

from utils.get_file import GetTV
from utils.processing import AdvancedProcessing, HomomorphicFilter

In [ ]:
tv = GetTV()
files = tv.list_files()
file = files[0]
vid = tv.load(file, 'vid')

In [ ]:
# single image plotting
idx = 245
kernel_size = 5
sigma = 1
threshold = 3
erode_kernel = 5
aspect_num = 1/2

process = AdvancedProcessing()
homofilt = HomomorphicFilter()

img = vid[idx]
probability = process.process_image(img)
# probability = homofilt.filter(img, (.9,50))

fig, ax = plt.subplots(2)
ax[0].imshow(probability, cmap='gray')
ax[1].imshow(img, cmap='gray')
x_left, x_right = ax[0].get_xlim()
y_low, y_high = ax[0].get_ylim()
ax[0].set_aspect(abs((x_right-x_left)/(y_low-y_high))*aspect_num)
ax[1].set_aspect(abs((x_right-x_left)/(y_low-y_high))*aspect_num)
ax[0].set_title(f'2X BR + 2DMFLD, Frame {idx}')
ax[1].set_title(f'Original, Frame {idx}')
plt.tight_layout()
plt.show()

In [ ]:
# animated plotting
savepath = Path('process_videos') / f'{filename}_inversion.mp4'

kernel_size = 5
sigma = 1
threshold = 4
erode_kernel = 4
aspect_num = 1/2

fig, ax = plt.subplots(2)
top = ax[0].imshow(probability, cmap='gray')
bot = ax[1].imshow(img8, cmap='gray')
title1 = ax[0].set_title(f'2X BR + 2DMFLD, Frame {0}')
title2 = ax[1].set_title(f'Original, Frame {0}')

x_left, x_right = ax[0].get_xlim()
y_low, y_high = ax[0].get_ylim()
ax[0].set_aspect(abs((x_right-x_left)/(y_low-y_high))*aspect_num)
ax[1].set_aspect(abs((x_right-x_left)/(y_low-y_high))*aspect_num)

plt.tight_layout()

def animate(i):
    img = np.sqrt(vid[i]).copy() # [25:250,250:700]
    ratio = np.amax(img) / 255
    img8 = (img/ratio).astype('uint8')
    img8 = img8[0:240, 240:720]
    probability = process_image(img8, kernel_size, sigma, threshold, erode_kernel)
    top.set_array(probability)
    bot.set_array(img8)
    title1.set_text(f'2X BR + 2DMFLD, Frame {i}')
    title2.set_text(f'Original, Frame {i}')
    return top, bot, title1, title2
    
writervideo = animation.FFMpegWriter(fps=30) 
ani = animation.FuncAnimation(fig, animate, frames=tqdm(range(len(vid))), blit=True)
ani.save(savepath, writer=writervideo)
plt.close()